In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
import json
import time
import sys
from selenium.webdriver.common.by import By
import undetected_chromedriver as uc
from fake_useragent import UserAgent
import pyperclip
from datetime import datetime
from copy import deepcopy

op = webdriver.ChromeOptions()
op.add_argument(f"user-agent={UserAgent.random}")
op.add_argument("user-data-dir=./") # op.add_argument("user-data-dir=./")
op.add_experimental_option("detach", True)
op.add_experimental_option("excludeSwitches", ["enable-logging"])

In [2]:
prompt_str_original = r"""
Using the episode summary, reply only in a format with valid JSON providing context for the dialogue exchanges. For each prompt/completion JSON provided, separate the new dialogue onto another line. Do not feel the need to summarize the episode itself in your response or per-dialogue. I only care about context that can help explain why the dialogue is funny. Supply the context in a way that makes it as funny as possible.

Here is an example for one that is funny:
1: {"prompt": {"identifier": 50, "episode": 20, "context": "Context for episode 20", "dialogue": [{"name": "GINTOKI", "text": "He's done for with that on him."}, {"name": "HIJIKATA", "text": "What are they doing? Can I stab these jokers? Huh, can I?"}, {"name": "KONDO", "text": "Sensei, can you help us? I'm too scared to go to the bathroom by myself."}, {"name": "KAGURA", "text": "We'll take care of it, gorilla."}, {"name": "KONDO", "text": "Huh? Did you just say \"gorilla\"? You just said gorilla, right?"}, {"name": "GINTOKI", "text": "<mask>"}]}, "completion": {"text": "We looked around the house. And I feel a fairly powerful spirit presence, gorilla."}}

The response would be as below. Note that you only include the number, a colon, the identifier, and then the context as seen below in valid JSON format. Do not stray from this format.
1: {"identifier": 50, "context": "KONDO, an adult man and leadership figure, humorously pretends to be a child by saying 'sensei' despite being the leader, as he is terrified of the ghosts. Kagura calls him a gorilla and he seems a bit slow to be offended, which is funny. Gintoki joins in further adding to the comedy."}

General Show Details:
Gintama is a comedy rife with parody, with frequent fourth wall breaks and referenced to real-life people, shows, and other characters from other shows. 

Episode Details:
"""

########### INPUTS
gpt_4 = False
run_chatgpt = True
###
use_episode_exception = True
episode_exception = 55 # when above is true, ONLY do this episode (good for automatically generating prompts for stuff that was skipped for whatever reason). Want to set run_chatgpt to false if so though.
###
# Initialize variables for tracking chunks and episodes
current_chunk = []
current_episode = None
episode_results = {}

In [3]:
################
# HOW TO RUN:
# cmd: chrome.exe --remote-debugging-port=9222 --user-data-dir="E:\Programs\AI Finetuning\Contexts\profile"
# just keep it open. Don't actually use it. Just need session to exist.
# run code + login + clear prompts + select GPT ver + type in 1 thing (wastes prompt, unlucky)

if run_chatgpt:
    driver = uc.Chrome(chrome_options=op)
    driver.get('https://chat.openai.com/auth/login')
else:
    for i in range(0, 245):
        if i in episode_results and (i != episode_exception and use_episode_exception):
            episode_results.pop(i)

In [4]:
with open("E:\Programs\AI Finetuning\Wikipedia Context\episode_scrape_cleaned.json", "r", encoding="utf-8") as f:
    wiki_dict = json.load(f)
    
# Define the filename and the number of items per chunk
filename = r"E:\Programs\AI Finetuning\Subs\Cleaned\finetuning_trimmed.jsonl"

mega_episodes = [100] # maybe just dialogue is way too long here?
large_episodes = [170, 168, 45, 166, 89, 69, 154, 72, 169, 66, 98, 88, 203, 99, 131, 95, 91, 228, 46, 92, 134, 121]
medium_episodes = [110, 184, 79, 96, 38, 155, 202, 216, 240, 167, 132, 229, 20, 53, 123, 237, 183, 122, 218, 225, 239, 226, 54, 90, 94]
smaller_episodes = [114, 34, 182, 25, 119, 238, 55, 133, 109, 48, 111, 165, 29, 37, 73, 120, 50, 150, 125]

chunk_size_dict = {}# make it per episode? some can get away with more prompts and some with less
for episode in (mega_episodes + large_episodes + medium_episodes + smaller_episodes):
    if episode in mega_episodes:
        chunk_size_dict[episode] = 2
    if episode in large_episodes:
        chunk_size_dict[episode] = 6
    elif episode in medium_episodes:
        chunk_size_dict[episode] = 8
    elif episode in smaller_episodes:
        chunk_size_dict[episode] = 10
        



# Open the file and iterate over the lines
with open(filename, 'r', encoding="utf-8") as f:
    for line in f:
        # Parse the JSON and extract the episode number
        data = json.loads(line)
        episode = data['prompt']['episode']

        chunk_size = chunk_size_dict[episode]
        if not gpt_4:
            chunk_size /= 2
        
        # If the episode changes or the chunk is full, append the current chunk to the episode results and start a new chunk
        if episode != current_episode or len(current_chunk) == chunk_size:
            if current_episode is not None:
                if current_episode not in episode_results:
                    episode_results[current_episode] = []
                episode_results[current_episode].append(current_chunk)
            current_chunk = []
            current_episode = episode
        
        # Append the current line to the current chunk
        current_chunk.append(data)
    
    # Append the final chunk to the episode results
    if current_episode not in episode_results:
        episode_results[current_episode] = []
    episode_results[current_episode].append(current_chunk)

In [ ]:
# Print the resulting episode results
k = 0
if gpt_4:
    first_pass = True
    first_pass_prompts = 24
    default_prompt_limit = 25
    wait_time = 60 * 150
    copy_paste_delay = 5
    message_write_delay = 80
else:
    first_pass = False
    first_pass_prompts = 100
    default_prompt_limit = 200
    wait_time = 60 * 150
    copy_paste_delay = 5
    message_write_delay = 50  # 1 dialogue ~ about 10 seconds without chatgpt plus

for episode, dialogue_list in episode_results.items():
    print(f"On episode: {episode}")
    for dialogues in dialogue_list:
        k += 1
        # print(episode, len(dialogues))
        episode_str = str(episode)

        episode_wiki = wiki_dict[episode_str]

        prompt_str = deepcopy(prompt_str_original) # maybe this is causing issues???
        prompt_str += (episode_wiki["Introduction"] + "\n")
        prompt_str += (episode_wiki["Plot"] + "\n")
        prompt_str += f"Below are {len(dialogues)} dialogue items separated by space to respind to. Please respond as indicated above by just writing the \"context\".\n"
        for i, dialogue in enumerate(dialogues):
            prompt_str += f"{i+1}: {dialogue}\n\n"

        # # print(prompt_str)
        # print(f"Copying prompt {k}")

        # # time.sleep(60)
        # input("Press to continue")
        # print("================================")
        if run_chatgpt:
            inputElements = driver.find_elements(By.TAG_NAME, "textarea")
            pyperclip.copy(prompt_str)
            time.sleep(copy_paste_delay)
            # inputElements[0].send_keys(prompt_str) # this is only producing partial output. Use copy/paste instead or something.
            try:
                inputElements[0].send_keys(Keys.CONTROL + 'v') # this is only producing partial output. Use copy/paste instead or something.
            except IndexError as e:
                print(e)
                print("Waiting 20 more seconds and then trying again.")
                inputElements = driver.find_elements(By.TAG_NAME, "textarea")
                pyperclip.copy(prompt_str)
                time.sleep(5)
                print("Failed on prompt:")
                print(prompt_str)

            time.sleep(copy_paste_delay)
            inputElements[0].send_keys(Keys.ENTER)
            time.sleep(message_write_delay)

            # temp cuz I'm an idiot
            if first_pass:
                bool = k % first_pass_prompts == 0
            else:
                bool = k % default_prompt_limit == 0

            if bool:
                first_pass = False
                
                now = datetime.now()
                current_time = now.strftime("%H:%M:%S")
                print("Current Time =", current_time)
                print("Waiting for 3 hours")
                time.sleep(60 * 150) # wait 155 minutes (180 - 25)
                get_url = driver.current_url # make sure that the page is returned to when refreshed
                time.sleep(5)
                driver.refresh()
                time.sleep(20) # wait 20 seconds.
                driver.get(get_url)  # make sure that the page is returned to when refreshed
                time.sleep(10) # wait 10 seconds.
        else:
            print(prompt_str)
            pyperclip.copy(prompt_str)
            print(f"Copying prompt")